# Playing with JS

In [1]:
// libraries
var fs = require('fs');
var d3 = require('../src/node_modules/d3');

// files
var f1 = '../data/indexed-courses.json'; // new line delimited
var f2 = '../data/test-courses.json'; // indexed

undefined

In [2]:
function read_courses(f) {
    var buffer = fs.readFileSync(f);
    var contents = JSON.parse(buffer);
    return contents;
}

undefined

In [3]:
var courses = read_courses(f2);

undefined

In [4]:
function create_course_graph(course) {
    var final_nodes = [],
        remaining_nodes = [],
        next_nodes = [];
    
    var traverse = [{linkType : "prereqs",
                      nodeType : "source"}];
    
    //
    traverse.forEach((step) => {
        
        //
        course[step.linkType].forEach((link) => {
            final_nodes.push(link[step.nodeType]);
            remaining_nodes.push(link[step.nodeType]);
        });
        
    });
    
    console.log(remaining_nodes, next_nodes);
}

undefined

In [5]:
create_course_graph(courses['CS:3330']);

[ 'CS:2210', 'MATH:1550', 'MATH:1850', 'CS:2230' ] []


undefined

In [15]:
function create_course_graph2(course, courses) {
    var final_nodes = new Set(),
        final_links = new Set(),
        remaining_nodes = [],
        next_nodes = [];
    
    var traverse = [{linkType : "prereqs",
                      nodeType : "source"}];
    
    //
    traverse.forEach((step) => {
        
        // initially add all the prereqs from a class 
        // to BOTH the final and remaining nodes
        course[step.linkType].forEach((link) => {
            // set to check against
            final_nodes.add(link[step.nodeType]);
            // links
            final_links.add(link);
            // list
            remaining_nodes.push(link[step.nodeType]);
        });
        
        // for each prereq, add it's prereqs
        remaining_nodes.forEach(function(node) {
            // get the course from it's ID
            node = courses[node];

            node[step.linkType].forEach(function(link) {

                var prereq_id = link[step.nodeType];
                if (final_nodes.has(prereq_id) == false) {
                    next_nodes.push(link[step.nodeType]);
                    final_nodes.add(prereq_id);
                    final_links.add(link);
                }
            });
            
        });      
        
    });
    
    console.log(remaining_nodes);
    console.log(next_nodes)
    console.log(final_nodes);
    console.log(final_links);
}

undefined

In [16]:
create_course_graph2(courses['CS:3330'], courses);

[ 'CS:2210', 'MATH:1550', 'MATH:1850', 'CS:2230' ]
[ 'MATH:1380',
  'MATH:1020',
  'MATH:1460',
  'MATH:1010',
  'MATH:1005',
  'MATH:1340',
  'CS:1210',
  'ENGR:2730' ]
Set {
  'CS:2210',
  'MATH:1550',
  'MATH:1850',
  'CS:2230',
  'MATH:1380',
  'MATH:1020',
  'MATH:1460',
  'MATH:1010',
  'MATH:1005',
  'MATH:1340',
  'CS:1210',
  'ENGR:2730' }
Set {
  { source: 'CS:2210', target: 'CS:3330', value: 1 },
  { source: 'MATH:1550', target: 'CS:3330', value: 1 },
  { source: 'MATH:1850', target: 'CS:3330', value: 1 },
  { source: 'CS:2230', target: 'CS:3330', value: 1 },
  { source: 'MATH:1380', target: 'MATH:1550', value: 1 },
  { source: 'MATH:1020', target: 'MATH:1550', value: 1 },
  { source: 'MATH:1460', target: 'MATH:1550', value: 1 },
  { source: 'MATH:1010', target: 'MATH:1550', value: 1 },
  { source: 'MATH:1005', target: 'MATH:1550', value: 1 },
  { source: 'MATH:1340', target: 'MATH:1550', value: 1 },
  { source: 'CS:1210', target: 'CS:2230', value: 1 },
  { source: 'ENGR:273

undefined

In [29]:
function create_course_graph3(course, courses) {
    var final_nodes = new Set(),
        final_links = new Set(),
        remaining_nodes = [],
        next_nodes = [];
    
    var traverse = [{linkType : "prereqs",
                      nodeType : "source"}];


    // initially add all the prereqs from a class 
    // to BOTH the final and remaining nodes
    course['prereqs'].forEach((link) => {
        // set to check against
        final_nodes.add(link['source']);
        // links
        final_links.add(link);
        // list
        remaining_nodes.push(link['source']);
    });

    // for each prereq, add it's prereqs
    remaining_nodes.forEach(function(node) {
        // get the course from it's ID
        node = courses[node];

        node['prereqs'].forEach(function(link) {

            var prereq_id = link['source'];
            if (final_nodes.has(prereq_id) == false) {
                next_nodes.push(link['source']);
                final_nodes.add(prereq_id);
                final_links.add(link);
            }
        });

    });      
    
    return {"nodes": Array.from(final_nodes), "links": Array.from(final_links)};
}

undefined

In [30]:
create_course_graph3(courses['CS:3330'], courses);

[ 'CS:2210', 'MATH:1550', 'MATH:1850', 'CS:2230' ]


{ nodes: 
   [ 'CS:2210',
     'MATH:1550',
     'MATH:1850',
     'CS:2230',
     'MATH:1380',
     'MATH:1020',
     'MATH:1460',
     'MATH:1010',
     'MATH:1005',
     'MATH:1340',
     'CS:1210',
     'ENGR:2730' ],
  links: 
   [ { source: 'CS:2210', target: 'CS:3330', value: 1 },
     { source: 'MATH:1550', target: 'CS:3330', value: 1 },
     { source: 'MATH:1850', target: 'CS:3330', value: 1 },
     { source: 'CS:2230', target: 'CS:3330', value: 1 },
     { source: 'MATH:1380', target: 'MATH:1550', value: 1 },
     { source: 'MATH:1020', target: 'MATH:1550', value: 1 },
     { source: 'MATH:1460', target: 'MATH:1550', value: 1 },
     { source: 'MATH:1010', target: 'MATH:1550', value: 1 },
     { source: 'MATH:1005', target: 'MATH:1550', value: 1 },
     { source: 'MATH:1340', target: 'MATH:1550', value: 1 },
     { source: 'CS:1210', target: 'CS:2230', value: 1 },
     { source: 'ENGR:2730', target: 'CS:2230', value: 1 } ] }

In [8]:
function create_course_graph_recursive(course, courses) {
    var final_nodes = new Set(),
        remaining_nodes = [],
        next_nodes = [];
    
    var traverse = [{linkType : "prereqs",
                      nodeType : "source"}];
    
    //
    traverse.forEach((step) => {
        
        // initially add all the prereqs from a class 
        // to BOTH the final and remaining nodes
        course[step.linkType].forEach((link) => {
            // set
            final_nodes.add(link[step.nodeType]);
            // list
            remaining_nodes.push(link[step.nodeType]);
        });
        
        
        while (remaining_nodes.length) {
            nextNodes = [];
            
            remaining_nodes.forEach(function(node) {
                // get the course from it's ID
                node = courses[node];
                
                /*
                node[step.linkType].forEach(function(link) {
                    
                    var prereq_id = link[step.nodeType];
                    if (final_nodes.has(prereq_id) == false) {
                       next_nodes.push(link[step.nodeType]);
                       final_nodes.add(prereq_id); 
                    }
                });
                */
            });
            
            //remaining_nodes = next_nodes;
        }
        
        
        
        
    });
    
    console.log(remaining_nodes, next_nodes, final_nodes);
}

undefined